In [13]:
'''>>>>> start CodeP3.2.1F24
    V.P. Carey ME249, Fall 2024 

Intro to Neural Network Modeling 
Keras model for PV power system'''

#import useful packages
import keras
import pandas as pd
from keras.models import Sequential
import numpy as np
import keras.backend as kb
import tensorflow as tf
#the following 2 lines are only needed for Mac OS machines
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# define meadian values of input variables - add your values here
Tamed = 1.  #make sure Tamed does not = 0
IDmed = 1.
RLmed = 1.

#create input data array
xdata = []

#Part 1 input data: Mode number, Air temp (degC), ID (W/sqm), load resistance (ohms)
xdata = [[0., 10.0, 200.0, 24.3]] 
xdata.append([1., 10.0, 200.0, 24.3]) 
xdata.append([2., 10.0, 200.0, 24.3])
xdata.append([1., 10.0, 200.0, 24.3])

''' ADD THE REST OF THE INPUT DATA FROM DS3.2.1multiModePerfF23 AND CONVERT TO:
xdata = [[0., 10.0/Tamed, 200.0/IDmed, 24.3/RLmed]] 
xdata.append([1., 10.0/Tamed, 200.0/IDmed, 24.3/RLmed]) 
xdata.append([2., 10.0/Tamed, 200.0/IDmed, 24.3/RLmed])
... etc.'''

xarray= np.array(xdata)
print (xdata)
print (xarray)

# define meadian values of output variables - add your values here
VLmed = 1.
Wdmed = 1

#create output data array
ydata = []

#Part 1 output data: VL (V) and Power out Wd (W)
ydata = [[46.0, 87.3]] 
ydata.append([27.9, 64.2]) 
ydata.append([27.9, 32.1]) 
ydata.append([46.0, 40.9]) 

''' ADD THE REST OF THE OUPUT DATA AND CONVERT TO:
ydata = [[46.0/VLmed, 87.3/Wdmed]] 
ydata.append([27.9/VLmed, 64.2/Wdmed]) 
ydata.append([27.9/VLmed, 32.1/Wdmed]) 
ydata.append([46.0/VLmed, 40.9/Wdmed])
... etc.'''

yarray= np.array(ydata)
print (ydata)
print (yarray)

[[0.0, 10.0, 200.0, 24.3], [1.0, 10.0, 200.0, 24.3], [2.0, 10.0, 200.0, 24.3], [1.0, 10.0, 200.0, 24.3]]
[[  0.   10.  200.   24.3]
 [  1.   10.  200.   24.3]
 [  2.   10.  200.   24.3]
 [  1.   10.  200.   24.3]]
[[46.0, 87.3], [27.9, 64.2], [27.9, 32.1], [46.0, 40.9]]
[[46.  87.3]
 [27.9 64.2]
 [27.9 32.1]
 [46.  40.9]]


In [17]:
# define neural network model

#As seen below, we have created four dense layers. 
#A dense layer is a layer in neural network that’s fully connected. 
#In other words, all the neurons in one layer are connected to all other neurons in the next layer.
#In the first layer, we need to provide the input shape, which is 1 in our case. 
#The activation function we have chosen is elu, which stands for exponential linear unit. .

from keras import backend as K
#initialize weights with values between -0.2 and 0.5
initializer = keras.initializers.RandomUniform(minval= -0.2, maxval=0.5)

model = keras.Sequential([
    keras.layers.Dense(13, activation=K.elu, input_shape=[4],  kernel_initializer=initializer),
    keras.layers.Dense(26, activation=K.elu,  kernel_initializer=initializer),
    keras.layers.Dense(13, activation=K.elu, kernel_initializer=initializer),
    keras.layers.Dense(2,  kernel_initializer=initializer)
  ])


In [18]:
#We’re using RMSprop as our optimizer here. RMSprop stands for Root Mean Square Propagation. 
#It’s one of the most popular gradient descent optimization algorithms for deep learning networks. 
#RMSprop is an optimizer that’s reliable and fast.
#We’re compiling the mode using the model.compile function. The loss function used here 
#is mean squared error. After the compilation of the model, we’ll use the fit method with ~500 epochs.
#Number of epochs can be varied.

#from tf.keras import optimizers
rms = keras.optimizers.RMSprop(0.020)
model.compile(loss='mean_absolute_error',optimizer=rms)


In [16]:
#After the compilation of the model, we’ll use the fit method with 500 epochs.
#I started with epochs value of 100 and then tested the model after training. 
#The prediction was not that good. Then I modified the number of epochs to 200 and tested the model again. 
#Accuracy had improved slightly, but figured I’d give it one more try. Finally, at 500 epochs 
#I found acceptable prediction accuracy.

#The fit method takes three parameters; namely, x, y, and number of epochs. 
#During model training, if all the batches of data are seen by the model once, 
#we say that one epoch has been completed.

# Add an early stopping callback
es = keras.callbacks.EarlyStopping(
    monitor='loss', 
    mode='min', 
    patience = 80, 
    restore_best_weights = True, 
    verbose=1)
# Add a checkpoint where loss is minimum, and save that model
mc = keras.callbacks.ModelCheckpoint('best_model.SB', monitor='loss', 
                     mode='min',  verbose=1, save_best_only=True)

historyData = model.fit(xarray,yarray,epochs=800,callbacks=[es])

loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1
print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))

model.save('./best_model')

Epoch 1/800


ValueError: in user code:

    File "/Users/vpcarey/anaconda3/envs/Python3-7_keras_env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/vpcarey/anaconda3/envs/Python3-7_keras_env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/vpcarey/anaconda3/envs/Python3-7_keras_env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/vpcarey/anaconda3/envs/Python3-7_keras_env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/vpcarey/anaconda3/envs/Python3-7_keras_env/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f

In [ ]:


# This line of code can be used to reconstruct the saved model.

recon_model = keras.models.load_model("best_model")

# the name of the model is now "recon_model". You can then use this model to do predictions for comparisons.
# See the previous project for code to do the comparisons.






